In [ ]:
import openai 
from tenacity import * 

import openai 
PATH_TO_KEY="/home/data1/pie_data_sept_2023/openai_key.txt"
with open(PATH_TO_KEY, 'r') as f:
    key = f.read().strip()
PATH_TO_ORG="/home/data1/pie_data_sept_2023/openai_org.txt"
with open(PATH_TO_ORG, 'r') as f:
    org = f.read().strip()
openai.organization = org
openai.api_key = key

In [ ]:
def print_model_list(): 
    response = openai.Model.list()

    # Extract and print the model names
    for model in response["data"]:
        print(model["id"])

In [ ]:
print_model_list()

In [ ]:
# model = "gpt-4o-2024-05-13"
model = "gpt-3.5-turbo-0125"

In [ ]:
# PATH_TO_OUTPUT = "/data1/shypula/Project_CodeNet/processed"
# if not os.path.exists(PATH_TO_OUTPUT):
#     os.makedirs(PATH_TO_OUTPUT)
# # save each to a jsonl file, records and lines = True 
# train_records_df = pd.DataFrame(train_records)
# test_records_df = pd.DataFrame(test_records)
# val_records_df = pd.DataFrame(val_records)

# train_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "train_descriptions_and_testcases.jsonl"), lines=True, orient="records")
# test_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "test_descriptions_and_testcases.jsonl"), lines=True, orient="records")
# val_records_df.to_json(os.path.join(PATH_TO_OUTPUT, "val_descriptions_and_testcases.jsonl"), lines=True, orient="records")

import pandas as pd 

train_records = pd.read_json("/home/data1/processed/train_descriptions_and_testcases.jsonl", lines=True)
test_records = pd.read_json("/home/data1/processed/test_descriptions_and_testcases.jsonl", lines=True)
val_records = pd.read_json("/home/data1/processed/val_descriptions_and_testcases.jsonl", lines=True)

all_records = pd.concat([train_records, test_records, val_records])
print(all_records.shape)

In [ ]:
# print(all_records["description_string"].iloc[0].replace("\n+", "\n"))
import re 
canonicalize_newlines = lambda x: re.sub(r"\n+", "\n", x)
all_records["description_string_canonical"] = all_records["description_string"].apply(canonicalize_newlines)
print(all_records["description_string_canonical"].iloc[0])

In [ ]:
import sys 

sys.path.append("/home/alex/Documents/PennPhD/program_cal_diverse") 

from utils import open_ended 

In [ ]:
results = open_ended.prompt_gpt_to_generate_input_description(
    all_records["description_string_canonical"].iloc[0],
    open_ended.example_problem, open_ended.example_input, model=open_ended.default_model, temperature=0.5, n=2, top_p=1.0)

In [ ]:
results

In [ ]:
results = open_ended.prompt_gpt_to_write_testcase_generator(
    all_records["description_string_canonical"].iloc[0],
    open_ended.example_problem, open_ended.example_tcgen, model=open_ended.default_model, temperature=0.5, n=2, top_p=1.0)


In [ ]:
results

In [ ]:
# re-import 
import importlib 
importlib.reload(open_ended)

In [ ]:
html_examle = open_ended.make_n_html_from_programs(
    all_records["description_string_canonical"].iloc[0],
    n=1, 
    temp=0.0,
    top_p=1.0,
    model=open_ended.default_model
)


In [ ]:
# from IPython.display import HTML
# HTML(html_examle[0][0])    

In [ ]:
import joblib 

In [ ]:
from joblib import Parallel, delayed
import tqdm
import contextlib
import os

@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

def process_problems_parallel(problems_with_ids, n, temp, top_p, model, output_directory):
    with tqdm_joblib(tqdm.tqdm(total=len(problems_with_ids))) as progress_bar:
        results = Parallel(n_jobs=10)(delayed(open_ended.make_n_html_from_programs)(
            desc, n, temp, top_p, model, problem_no=problem_id
        ) for problem_id, desc in problems_with_ids)
        
        for (problem_id, _), (html_outputs, input_descriptions, tc_gens, extract_args) in zip(problems_with_ids, results):
            problem_output_dir = os.path.join(output_directory, f"problem_{problem_id}")
            save_outputs_to_files(problem_id, problem_output_dir, html_outputs, input_descriptions, tc_gens, extract_args)
            progress_bar.update(1)  # Ensure the progress bar updates correctly per problem processed

def save_outputs_to_files(problem_id, output_directory, html_outputs, input_descriptions, tc_gens, extract_args):
    os.makedirs(output_directory, exist_ok=True)
    
    for i in range(len(html_outputs)):
        generation = i + 1
        html_file = os.path.join(output_directory, f"html_{problem_id}_{generation}.html")
        input_desc_file = os.path.join(output_directory, f"input_desc_{problem_id}_{generation}.txt")
        tc_gen_file = os.path.join(output_directory, f"tc_gen_{problem_id}_{generation}.txt")
        extract_args_file = os.path.join(output_directory, f"extract_args_{problem_id}_{generation}.txt")
        
        with open(html_file, 'w') as file:
            file.write(html_outputs[i])
        with open(input_desc_file, 'w') as file:
            file.write(input_descriptions[i])
        with open(tc_gen_file, 'w') as file:
            file.write(tc_gens[i])
        with open(extract_args_file, 'w') as file:
            file.write(extract_args[i])
            


# Example usage
# problems_with_ids = [('1', 'Some description 1'), ('2', 'Some description 2')]
# process_problems_parallel(problems_with_ids, 5, 0.8, 0.9, 'your_model_instance', '/path/to/output')


In [ ]:
all_records.columns

In [ ]:
# re-import 
import os 
import importlib 
import joblib 
importlib.reload(open_ended)

n_examples = 75
n_records = all_records.sample(n_examples)
descriptions = n_records["description_string_canonical"].tolist()
problem_ids = n_records["codenet_problem_id"].tolist()
problems_with_ids = list(zip(problem_ids, descriptions))
output_dir = "/home/data1/processed/outputs_v6_4o"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# process_problems_parallel(problems_with_ids, 3, 0.5, 1.0, open_ended.default_model, output_dir)
process_problems_parallel(problems_with_ids, 3, 0.75, 1.0, "gpt-4o-2024-05-13", output_dir)



In [ ]:
open_ended_final_path = "/home/data1/processed/open_ended_final/"
# read in the annotated notes and copy from /processed/output_* to /processed/open_ended_final based on columns 
import pandas as pd 
import shutil 

annotated_notes = pd.read_csv("/home/data1/processed/open_ended_final/examples_for_open_ended_gen_program_cal_diverse.csv")
# Index(['problem_id', 'specification_no', 'tcgen_no', 'Extract No',
#        'Additional Notes', 'Subjective Score (1-10)', 'Use? ',
#        'Specification'],
#       dtype='object')
annotated_notes = annotated_notes[annotated_notes["Use? "] == True]

annotated_notes.sort_values(by="Subjective Score (1-10)", ascending=False, inplace=True)
# reset index
annotated_notes = annotated_notes.reset_index(drop=True)
# cast the spec_no, tcgen, extract cols to int
for col in ["specification_no", "tcgen_no", "Extract No"]:
    annotated_notes[col] = annotated_notes[col].astype(int)
annotated_notes



In [ ]:
from tqdm import tqdm
import os 
def clean_string(input_string):
    # Replace non-breaking space with a regular space (or remove it entirely, if you prefer)
    cleaned_string = input_string.replace(u"\xa0", "").strip()
    return cleaned_string
for i, row in tqdm(annotated_notes.iterrows(), total=annotated_notes.shape[0]):
    problem_id = clean_string(row["problem_id"])
    spec_no = row["specification_no"]
    tcgen_no = row["tcgen_no"]
    extract_args = row["Extract No"]
    orig_dir = f"/home/data1/processed/outputs_v6_4o/problem_{problem_id}"
    dest_dir = f"{open_ended_final_path}/problem_{problem_id}"
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    # copy only the files that match the spec_no, tcgen_no, and extract_args
    shutil.copy2(clean_string(f"{orig_dir}/input_desc_{problem_id}_{spec_no}.txt"), f"{dest_dir}/input_desc_orig.txt")
    if not os.path.exists(f"{dest_dir}/input_desc_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/input_desc_{problem_id}_{spec_no}.txt"), f"{dest_dir}/input_desc_final.txt")
    # copy the tc_gen
    shutil.copy2(clean_string(f"{orig_dir}/tc_gen_{problem_id}_{tcgen_no}.txt"), f"{dest_dir}/tc_gen_orig.txt")
    if not os.path.exists(f"{dest_dir}/tc_gen_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/tc_gen_{problem_id}_{tcgen_no}.txt"), f"{dest_dir}/tc_gen_final.txt")
    # copy the extract_args
    shutil.copy2(clean_string(f"{orig_dir}/extract_args_{problem_id}_{extract_args}.txt"), f"{dest_dir}/extract_args_orig.txt")
    if not os.path.exists(f"{dest_dir}/extract_args_final.txt"):
        shutil.copy2(clean_string(f"{orig_dir}/extract_args_{problem_id}_{extract_args}.txt"), f"{dest_dir}/extract_args_final.txt")
    
    # also copy to "final"
    
    try: 
        orig_desc = all_records[all_records["codenet_problem_id"] == problem_id]["description_string_canonical"].iloc[0]
        with open(f"{dest_dir}/description.txt", 'w') as file:
            file.write(orig_desc)
    except Exception as e:
        print(f"Error with problem_id {problem_id}: {e}")
        

    

    
    
    

In [ ]:
all_records[all_records["codenet_problem_id"] == "p00050"]["description_string_canonical"].iloc[0]

In [ ]:
all_records[all_records["codenet_problem_id"] == "p00229"]

In [22]:

finalzed = {'p00050': True,
 'p00177': True,
 'p00229': True,
 'p00361': True,
 'p01020': True,
 'p01257': True,
 'p01508': True,
 'p01545': True,
 'p01639': True,
 'p02051': True,
 'p02181': True,
 'p02280': True,
 'p02731': True,
 'p02890': True,
 'p03222': True,
 'p03260': True,
 'p03666': True,
 'p03728': True,
 'p03774': True,
 'p03843': True,
 'p03907': True}

In [4]:
import random 

def tcgen_00361(): 
    N = random.randint(1, 10**4)  # Number of cities
    # 90% chance it is below 100 
    if random.random() < 0.9:
        M = random.randint(0, 100)
    else: 
        M = random.randint(0, 10**5)  # Number of roads

    roads = set()
    while len(roads) < M:
        s = random.randint(0, N-1)
        t = random.randint(0, N-1)
        if s != t:
            roads.add((s, t))
    
    roads = list(roads)
    
    test_case = f"{N} {len(roads)}\n"
    for s, t in roads:
        test_case += f"{s} {t}\n"
    
    return test_case.strip()

def make_testcase_01639():
        # Generating n, k, and x0 based on given constraints
        n = random.randint(1, 10**8)  # 1 ≤ n ≤ 10^8
        k = random.randint(1, n)  # 1 ≤ k ≤ n
        x0 = random.randint(0, 9)  # 0 ≤ x0 ≤ 9
        
        return n, k, x0

def tcgen_01639(): 
    n, k, x0 = make_testcase_01639()
    return f"{n} {k} {x0}"



def generate_binary_tree(n):
    """ Generates a random binary tree with n nodes. """
    nodes = list(range(n))
    random.shuffle(nodes)
    
    tree = {i: (-1, -1) for i in nodes}
    
    for i in range(n):
        if 2 * i + 1 < n:
            tree[nodes[i]] = (nodes[2 * i + 1], tree[nodes[i]][1])
        if 2 * i + 2 < n:
            tree[nodes[i]] = (tree[nodes[i]][0], nodes[2 * i + 2])

    return tree

def tcgen_02280():
    n = random.randint(1, 25)
    tree = generate_binary_tree(n)
    
    result = []
    # result.append(str(n))

    for node in sorted(tree.keys()):
        left, right = tree[node]
        result.append(f"{node} {left} {right}")
    
    output_str = f"{n}\n" + "\n".join(result)
    return output_str


tc2tcgen = {
    "p00361": tcgen_00361,
    "p01639": tcgen_01639, 
    "p02280": tcgen_02280
}



open_ended_final_path = "/home/data1/processed/open_ended_final/"
PATH_TO_TESTCASES = "/home/alex/Documents/PennPhD/learning2perf/data/codenet/merged_test_cases"

# read in the annotated notes and copy from /processed/output_* to /processed/open_ended_final based on columns 
import pandas as pd 
import shutil 
import os 
import re 
import glob

from tqdm import tqdm
dataset = []
for problem_id, use in tqdm(finalzed.items()):
    if use:
        record = {}
        ## get the open-ended final files
        directory_path = os.path.join(open_ended_final_path, f"problem_{problem_id}")
        input_desc_final = os.path.join(directory_path, "input_desc_final.txt")
        extract_args_final = os.path.join(directory_path, "extract_args_final.txt")
        tc_gen_final = os.path.join(directory_path, "tc_gen_final.txt")
        with open(input_desc_final, "r") as f:
            input_desc = f.read().strip()
        with open(extract_args_final, "r") as f:
            extract_args = f.read().strip()
        with open(tc_gen_final, "r") as f:
            tc_gen = f.read().strip()
        record["problem_id"] = problem_id
        record["description_string"] = input_desc
        record["extract_args_fun"] = extract_args
        record["tc_gen_fun"] = tc_gen
        # grab the testcases
        path_to_tcs = os.path.join(PATH_TO_TESTCASES, problem_id)
        input_files = glob.glob(os.path.join(path_to_tcs, "input.*.txt"))
        output_files = glob.glob(os.path.join(path_to_tcs, "output.*.txt"))
        num_input_files = len(input_files)
        tc2input = {}
        tc2output = {}
        if num_input_files > 10: 
            input_tc_nos = [re.search(r"input\.(\d+)\.txt", f).group(1) for f in input_files]
            for input_tc_no in input_tc_nos:
                input_tc_path = os.path.join(path_to_tcs, f"input.{input_tc_no}.txt")
                output_tc_path = os.path.join(path_to_tcs, f"output.{input_tc_no}.txt")
                assert os.path.exists(input_tc_path)
                assert os.path.exists(output_tc_path), f"Missing output file for {input_tc_path}"
                with open(input_tc_path, "r") as f:
                    input_tc = f.read().strip()
                with open(output_tc_path, "r") as f:
                    output_tc = f.read().strip()
                tc2input[input_tc_no] = input_tc
                tc2output[input_tc_no] = output_tc
        elif problem_id in tc2tcgen:
            print(f"Generating testcases for {problem_id}")
            tc_gen_fun = tc2tcgen[problem_id]
            for i in tqdm(range(100), desc="Generating testcases", mininterval=1, maxinterval=1):
                input_tc = tc_gen_fun()
                output_tc = "NA"
                tc2input[str(i)] = input_tc
                tc2output[str(i)] = output_tc
                if (i % 10) == 0:
                    print(f"Generated {i} testcases")
            num_input_files = 100
        else:
            raise ValueError(f"No testcases or tcgen function for {problem_id}")
        record["problem_id"] = problem_id
        record["input_testcases"] = tc2input
        record["output_testcases"] = tc2output
        record["num_input_files"] = num_input_files
        dataset.append(record)

dataset_df = pd.DataFrame(dataset)
dataset_df.to_json("/home/data1/processed/open_ended_final/dataset_update_v2.jsonl", lines=True, orient="records")

        
        
        
        
        



  0%|          | 0/21 [00:00<?, ?it/s]

Generating testcases for p00361


Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases


 19%|█▉        | 4/21 [00:00<00:01, 10.41it/s]

Generated 70 testcases
Generated 80 testcases
Generated 90 testcases
Generating testcases for p01639


Generating testcases: 100%|██████████| 100/100 [00:00<00:00, 526922.61it/s]


Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases
Generated 70 testcases
Generated 80 testcases
Generated 90 testcases
Generating testcases for p02280


100%|██████████| 21/21 [00:00<00:00, 51.14it/s]

Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases
Generated 70 testcases
Generated 80 testcases
Generated 90 testcases


In [5]:
dataset_df = pd.read_json("/home/data1/processed/open_ended_final/dataset.jsonl", lines=True)

In [6]:
# summarize number of input files
dataset_df["num_input_files"].describe()

count     21.000000
mean      99.523810
std       14.881596
min       35.000000
25%      102.000000
50%      102.000000
75%      104.000000
max      107.000000
Name: num_input_files, dtype: float64

In [7]:
# dataset_df[["num_input_files"], ["problem_id"]].sort_values(by="num_input_files", ascending=False)
dataset_df.sort_values(by="num_input_files", ascending=False)


,problem_id,description_string,extract_args_fun,tc_gen_fun,input_testcases,output_testcases,num_input_files
14,p03222,### Input Description:\n\nAn integer \(H\) (1 ...,"def extract_arguments(fh):\n H, W, K = map(...","def tcgen():\n H = random.randint(1, 100) ...","{'12': '2 6 1', '104': '2 3 3', '52': '20 8 1'...","{'12': '89', '104': '1', '52': '370317936', '2...",107
16,p03666,### Input Description:\n\nAn integer N (3 ≤ N ...,def extract_arguments(fh):\n # Read the lin...,import random\n\ndef tcgen():\n # Generate ...,{'12': '1788 122437383 943846071 90629745 1485...,"{'12': 'YES', '104': 'YES', '52': 'NO', '28': ...",105
19,p03843,### Input Description:\n\n1. An integer \(N\) ...,Certainly! Let's first write the `extract_argu...,## Note! not fully human checked\nimport rando...,"{'12': '4 8 4 2 3 2 4 1100', '52': '4 1 3 4 6 ...","{'12': '1', '52': '3', '28': '1', '84': '1', '...",104
18,p03774,### Input Description:\n\nAn integer \(N\) (1 ...,"def extract_arguments(fh):\n N, M = map(int...",import random\n\ndef make_testcase():\n # C...,"{'12': '3 4 10 6 -10 -10 3 4 0 4 2 3 3 9 3 5',...","{'12': '4 1 4', '52': '5 4 2 1 1', '28': '1 2 ...",104
17,p03728,### Input Description:\n\nAn integer \( N \) (...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef generate_permutation(n):\...,"{'12': '5 5 2 3 2 1', '52': '10 2 10 5 7 3 3 4...","{'12': '3', '52': '6', '28': '3', '84': '3', '...",104
15,p03260,### Input Description:\n\nAn integer A (1 ≤ A ...,"def extract_arguments(fh):\n A, B = map(int...",import random\n\ndef tcgen(): \n A = random...,"{'12': '-1 1', '52': '-2 -3', '28': '-2 -2', '...","{'12': 'Yes', '52': 'No', '28': 'No', '84': 'N...",104
13,p02890,### Canonicalized Input Description:\n\n1. An ...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Generate ...,"{'12': '4 2 3 1 3', '52': '4 1 3 4 3', '28': '...","{'12': '4 2 1 0', '52': '4 2 1 0', '28': '4 1 ...",104
20,p03907,### Input Description:\n\nAn integer \(N\) (2 ...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Step 1: G...,"{'12': '4 1 2 2 2 -1 1 -1 2', '52': '4 0 0 0 0...","{'12': '-1', '52': '-1', '28': '-1', '84': '-1...",103
12,p02731,### Input Description:\n\nAn integer \( L \) (...,def extract_arguments(fh):\n L = int(fh.rea...,import random\n\ndef tcgen():\n # Generate ...,"{'12': '-3', '52': '-22', '28': '19', '84': '1...","{'12': '-1.0', '52': '-394.37037037', '28': '2...",103
4,p01020,### Input Description:\n\n1. An integer \( N \...,def extract_arguments(fh):\n N = int(fh.rea...,import random\n\ndef tcgen():\n # Set the n...,"{'12': '4 5 0 -2 8', '52': '5 2 -4 0 3 2', '28...","{'12': '6', '52': '-5', '28': '42', '84': '7',...",103


In [11]:
print(dataset_df[dataset_df["problem_id"] == "p01639"].iloc[0]["input_testcases"])
print(dataset_df[dataset_df["problem_id"] == "p01639"].iloc[0]["description_string"])


{'0': '42997399 20423435 2', '1': '84178519 35935250 8', '2': '38628289 20628831 3', '3': '82271175 75257574 1', '4': '66989838 44688341 9', '5': '3345116 294240 6', '6': '68668823 63213731 1', '7': '67204085 16358191 5', '8': '31987491 1335156 3', '9': '38340470 34999201 6', '10': '91607248 77211987 6', '11': '49129621 19679977 9', '12': '70206569 63643164 7', '13': '5996443 2098766 3', '14': '76222240 35707481 1', '15': '73297984 44350395 6', '16': '69054252 29015835 8', '17': '18016899 15447602 2', '18': '32350378 28979102 5', '19': '9156342 5168910 1', '20': '23507829 8614551 5', '21': '69023272 1275295 6', '22': '77247199 18195238 9', '23': '22593652 20935110 8', '24': '46398014 34714122 5', '25': '87897621 22475514 4', '26': '64785804 4718550 6', '27': '89936350 68211718 9', '28': '39395656 10093020 3', '29': '48729733 22412284 2', '30': '57810927 38630098 9', '31': '58240552 53188619 0', '32': '68105606 13659505 8', '33': '62898731 21957897 6', '34': '64142012 9199435 6', '35': 

In [3]:
import pandas as pd 

data_df = pd.read_json("/home/shypula/program_cal_diverse/data/open_ended_final/open_ended_final/dataset.jsonl", lines=True, orient="records")
problematic_problem_ids = ["p01508", "p02051", "p03843"] 
data_df = data_df[data_df["problem_id"].isin(problematic_problem_ids)]
data_df 

,problem_id,description_string,extract_args_fun,tc_gen_fun,input_testcases,output_testcases,num_input_files
6,p01508,### Input Description:\n\nMultiple test cases....,def extract_arguments(fh):\n # Read all lin...,import random\nimport math\n\ndef generate_pol...,"{'12': '4 0 -1 10 0 3 10 0 7 3 1 0 2 0 0 1 0',...",{'12': 'Case 1: 12.2065556157 Case 2: 2.236067...,102
9,p02051,### Input Description\n\n1. An integer \( N \)...,#1. **Extract Arguments Function:**\n# - Rea...,import random\n\ndef tcgen():\n # Number of...,{'12': '4 9 1 2 1 1 2 2 1 3 1 2 3 2 1 3 3 2 3 ...,{'12': '1 9 42 19 1 2469133 457123934 82840427...,102
19,p03843,### Input Description:\n\n1. An integer \(N\) ...,Certainly! Let's first write the `extract_argu...,## Note! not fully human checked\nimport rando...,"{'12': '4 8 4 2 3 2 4 1100', '52': '4 1 3 4 6 ...","{'12': '1', '52': '3', '28': '1', '84': '1', '...",104


In [33]:
# with open("/home/shypula/program_cal_diverse/utils/clustering/open_ended_wrapper.py", "r") as f:
#     wrapper_code = f.read() 
    

def format_open_ended_code(f_code: str, extract_arguments_code: str) -> str:
    """
    Call this function to format the open-ended code with the f() function and extract_arguments() function
    for the open-ended scenario, to then be evaluated in the Docker container.
    """
    with open("/home/shypula/program_cal_diverse/utils/clustering/open_ended_wrapper.py", "r") as f:
        wrapper_code = f.read()
    formatted_wrapper = wrapper_code.replace("## REPLACE F", f_code).replace("## REPLACE EXTRACT_ARGUMENTS", extract_arguments_code)
    assert "## REPLACE F" not in formatted_wrapper, "F not replaced in formatted wrapper"
    assert "## REPLACE EXTRACT_ARGUMENTS" not in formatted_wrapper, "extract_arguments not replaced in formatted wrapper"
    return formatted_wrapper




In [48]:


## p01508
# description_string = data_df[data_df["problem_id"] == "p01508"].iloc[0]["description_string"]
# extract_args = data_df[data_df["problem_id"] == "p01508"].iloc[0]["extract_args_fun"]
# testcases = data_df[data_df["problem_id"] == "p01508"].iloc[0]["input_testcases"]

## p02051
# description_string = data_df[data_df["problem_id"] == "p02051"].iloc[0]["description_string"]
# extract_args = data_df[data_df["problem_id"] == "p02051"].iloc[0]["extract_args_fun"]
# testcases = data_df[data_df["problem_id"] == "p02051"].iloc[0]["input_testcases"]
# p03843

description_string = data_df[data_df["problem_id"] == "p03843"].iloc[0]["description_string"]
extract_args = data_df[data_df["problem_id"] == "p03843"].iloc[0]["extract_args_fun"]
testcases = data_df[data_df["problem_id"] == "p03843"].iloc[0]["input_testcases"]


print(description_string)
print('************')
print(extract_args)
print('************')
print(testcases['0'])

### Input Description:

1. An integer \(N\) (2 ≤ \(N\) ≤ 2×10^5), representing some quantity or size.
2. \(N-1\) pairs of integers (a_i, b_i) (1 ≤ a_i, b_i ≤ N), representing connections between entities.
3. A string \(s\) of length \(N\) consisting of characters '0' and '1'.

### Example Input:

```
4
1 2
1 3
1 4
1100
```

### Function Signature:

Write a function `f(N, connections, s)` that takes in the input.
```python
def f(N: int, connections: List[Tuple[int, int]], s: str):
    ''' 
    N: an integer 
    connections: a list of tuples, each containing two integers
    s: a string of '0's and '1's
    '''
```
************
Certainly! Let's first write the `extract_arguments` function for the provided problem statement, and then we'll write the function that processes these arguments.

### Extract Arguments Function

```python
def extract_arguments(fh):
    N = int(fh.readline().strip())
    edges = []
    for _ in range(N - 1):
        a, b = map(int, fh.readline().strip().split())

In [51]:
extract_args = """
def extract_arguments(fh):
    N = int(fh.readline().strip())
    edges = []
    for _ in range(N - 1):
        a, b = map(int, fh.readline().strip().split())
        edges.append((a, b))
    s = fh.readline().strip()
    return N, edges, s
"""

In [52]:
dummy_f = """
def f(N: int, connections: List[Tuple[int, int]], s: str):
    print("0")"""
wrapper = format_open_ended_code(dummy_f, extract_args)
for i, line in enumerate(wrapper.split("\n")):
    print(f"{i}: {line}")

0: ## TEMPLATE FOR OPEN-ENDED EXECUTION
1: from typing import * 
2: from typing import List, Tuple, Dict, Any, Union, Optional, Iterable, Callable, Mapping, TypeVar, Generic
3: import sys
4: import os
5: import resource
6: import sys
7: 
8: def limit_memory(maxsize):
9:     # Set maximum virtual memory to maxsize bytes
10:     soft, hard = resource.getrlimit(resource.RLIMIT_AS)
11:     resource.setrlimit(resource.RLIMIT_AS, (maxsize, hard))
12: 
13: try:
14:     # Example: Limit virtual memory to 1GB
15:     limit_memory(1024 * 1024 * 1024)
16: except ValueError:
17:     print("Error setting memory limit. Might require elevated privileges.")
18:     raise
19: 
20: # Your Python code here
21: 
22: 
23: from typing import TextIO
24: 
25: """
26: high level idea for f and extract_arguments: 
27: 
28: def f(...):
29:     pass
30:     
31: def extract_arguments(fh: TextIO) -> Tuple:
32:     N = int(fh.readline().strip())
33:     A = list(map(int, fh.readline().strip().split()))
34:     C = 

In [39]:
with open("/data1/shypula/prog_diversity/open_ended_v2/meta-llama-Meta-Llama-3-8B-Instruct_temp_1.0_top_p_1.0_max_length_1500_num_return_sequences_100_repetition_penalty_1.0_open_ended_default_2024-06-16_03-43-16/problem_p02051/formatted_prog_4_coh_0.0.txt", "r") as f:
    formatted_prog = f.read()

In [53]:

import subprocess
from tqdm import tqdm    
for tc_index, tc_no in tqdm(enumerate(testcases.keys()), total=len(testcases)):
    input_tc = testcases[tc_no]
    with open("/home/shypula/program_cal_diverse/input_tc.temp.txt", "w") as f:
        f.write(input_tc)
    with open("/home/shypula/program_cal_diverse/temp_exec.py", "w") as f:
        f.write(wrapper)
    p = subprocess.run(["python", "/home/shypula/program_cal_diverse/temp_exec.py", "/home/shypula/program_cal_diverse/input_tc.temp.txt"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        print(f"Error for {tc_no}")
        print(p.stderr.decode())
        break
    

100%|██████████| 104/104 [00:04<00:00, 22.75it/s]


In [32]:
p01508_fixed_extract = '''
def extract_arguments(fh):
    # Read all lines from the file handle
    lines = fh.readlines()
    
    # Initialize an empty list to store all test cases
    test_cases = []
    
    index = 0
    case_number = 1
    
    while index < len(lines):
        # Read the number of vertices
        N = int(lines[index].strip())
        
        if N == 0:
            break
        
        index += 1
        vertices = []
        
        # Read the N vertices
        for _ in range(N):
            x, y = map(int, lines[index].strip().split())
            vertices.append((x, y))
            index += 1
        
        # Append the extracted test case to the list
        test_cases.append((N, vertices))
        case_number += 1
    
    return test_cases
'''

In [47]:
p02051_fixed_desc = """
### Input Description

1. An integer \( N \) (1 ≤ \( N \) ≤ 2000), representing some quantity or size.
2. An integer \( M \) (1 ≤ \( M \) ≤ 2000), representing another quantity or size.
3. \( M \) lines follow, each containing three integers \( a_i \), \( b_i \), and \( c_i \) where \( 1 ≤ a_i < b_i ≤ N \) and \( 1 ≤ c_i ≤ 3 \).
4. An integer \( Q \) (1 ≤ \( Q \) ≤ 2000), representing yet another quantity or size.
5. \( Q \) lines follow, each containing two integers \( d_j \) and \( e_j \) where \( 1 ≤ d_j ≤ N \) and \( 0 ≤ e_j ≤ 100000 \).

### Example Input:

```
4 3
1 3 2
2 4 1
1 2 3
2
4 6
2 4
```

### Function Signature:
Write a function `f(inputs)` that takes in the input as a list of tuples.
```python
def f(N: int, M: int, triple: List[Tuple[int, int, int]], Q: int, pairs: List[Tuple[int, int]]):
    '''
    inputs: a list of tuples where each tuple contains:
        - an integer N
        - an integer M
        - a list of M tuples, each containing three integers (a_i, b_i, c_i)
        - an integer Q
        - a list of Q tuples, each containing two integers (d_j, e_j)
    '''
"""

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_2424803/2974636981.py:1: SyntaxWarning: invalid escape sequence '\('
  p02051_fixed_desc = """


In [54]:
p03843_fixed_extract_args = """
def extract_arguments(fh):
    N = int(fh.readline().strip())
    edges = []
    for _ in range(N - 1):
        a, b = map(int, fh.readline().strip().split())
        edges.append((a, b))
    s = fh.readline().strip()
    return N, edges, s
"""


In [54]:

import os 
import pandas as pd
from tqdm import tqdm 
import random 
data_df = pd.read_json("/home/shypula/program_cal_diverse/data/open_ended_final/open_ended_final/dataset.jsonl", lines=True, orient="records")
problematic_problem_ids = ["p01508", "p02051", "p03843", "p02280"]
data_dir = "/home/shypula/program_cal_diverse/data/open_ended_final"



# for 01508 replace the extract_args
# for 02051 replace the description
# for 03843 replace the extract_args
# for 02280 replace the input test cases 
# read from the updated *_final.txt and assert they are different 
def generate_binary_tree(n):
    """ Generates a random binary tree with n nodes. """
    nodes = list(range(n))
    random.shuffle(nodes)
    
    tree = {i: (-1, -1) for i in nodes}
    
    for i in range(n):
        if 2 * i + 1 < n:
            tree[nodes[i]] = (nodes[2 * i + 1], tree[nodes[i]][1])
        if 2 * i + 2 < n:
            tree[nodes[i]] = (tree[nodes[i]][0], nodes[2 * i + 2])

    return tree

def tcgen_02280():
    n = random.randint(1, 25)
    tree = generate_binary_tree(n)
    
    result = []
    # result.append(str(n))

    for node in sorted(tree.keys()):
        left, right = tree[node]
        result.append(f"{node} {left} {right}")
    
    output_str = f"{n}\n" + "\n".join(result)
    return output_str



for problem_id in problematic_problem_ids:
    directory_path = os.path.join(data_dir, f"problem_{problem_id}")
    input_desc_final = os.path.join(directory_path, "input_desc_final.txt")
    extract_args_final = os.path.join(directory_path, "extract_args_final.txt")
    tc_gen_final = os.path.join(directory_path, "tc_gen_final.txt")
    with open(input_desc_final, "r") as f:
        input_desc = f.read().strip()
    with open(extract_args_final, "r") as f:
        extract_args = f.read().strip()
    with open(tc_gen_final, "r") as f:
        tc_gen = f.read().strip()
    if problem_id == "p01508":
        assert data_df[data_df["problem_id"] == problem_id].iloc[0]["extract_args_fun"] != extract_args, "Extract args for 01508 not fixed"
        # data_df.loc[data_df["problem_id"] == problem_id, "extract_args_fun"] = p01508_fixed_extract
        data_df.loc[data_df["problem_id"] == problem_id, "extract_args_fun"] = extract_args
    elif problem_id == "p02051":
        # assert data_df[data_df["problem_id"] == problem_id].iloc[0]["description_string"] != p02051_fixed_desc, "Description for 02051 not fixed"
        assert data_df[data_df["problem_id"] == problem_id].iloc[0]["description_string"] != input_desc, "Description for 02051 not fixed"
        # data_df.loc[data_df["problem_id"] == problem_id, "description_string"] = p02051_fixed_desc
        data_df.loc[data_df["problem_id"] == problem_id, "description_string"] = input_desc
    elif problem_id == "p03843":
        # assert data_df[data_df["problem_id"] == problem_id].iloc[0]["extract_args_fun"] != p03843_fixed_extract_args, "Extract args for 03843 not fixed"
        assert data_df[data_df["problem_id"] == problem_id].iloc[0]["extract_args_fun"] != extract_args, "Extract args for 03843 not fixed"
        # data_df.loc[data_df["problem_id"] == problem_id, "extract_args_fun"] = p03843_fixed_extract_args
        data_df.loc[data_df["problem_id"] == problem_id, "extract_args_fun"] = extract_args
    elif problem_id == "p02280":
        tc2input = {}
        tc2output = {}
        assert "p02280" in data_df["problem_id"].values
        indices = data_df[data_df['problem_id'] == 'p02280'].index
        print("Indices found:", indices)
        if  indices.empty:
            raise ValueError("No indices found")
        # Use the .iat[] or .at[] for a more direct assignment to the first found index
        # data_df.at[indices[0], 'input_testcases'] = tc2input
        # print("Assigned Value:", data_df.at[indices[0], 'input_testcases'])

        for i in tqdm(range(100), desc="Generating testcases", mininterval=1, maxinterval=1):
            input_tc = tcgen_02280()
            output_tc = "NA"
            tc2input[str(i)] = input_tc
            tc2output[str(i)] = output_tc
            if (i % 10) == 0:
                print(f"Generated {i} testcases")
        num_input_files = 100
        # import pdb; pdb.set_trace()
        # data_df.loc[data_df["problem_id"] == problem_id, "input_testcases"] = tc2input
        # data_df.loc[data_df["problem_id"] == problem_id, "output_testcases"] = tc2output
        # data_df.loc[data_df["problem_id"] == problem_id, "num_input_files"] = 100
        data_df.at[indices[0], 'input_testcases'] = tc2input
        data_df.at[indices[0], 'output_testcases'] = tc2output
        data_df.at[indices[0], 'num_input_files'] = 100
        # print(data_df[data_df["problem_id"] == problem_id].iloc[0]["input_testcases"])
        
        
    else:
        raise ValueError(f"Unrecognized problem_id: {problem_id}")
    # update data_df 
    
data_df.to_json("/home/shypula/program_cal_diverse/data/open_ended_final/dataset_update.jsonl", lines=True, orient="records")
    

Indices found: Index([11], dtype='int64')


Generating testcases: 100%|██████████| 100/100 [00:00<00:00, 57733.02it/s]

Generated 0 testcases
Generated 10 testcases
Generated 20 testcases
Generated 30 testcases
Generated 40 testcases
Generated 50 testcases
Generated 60 testcases
Generated 70 testcases
Generated 80 testcases
Generated 90 testcases


In [51]:
data_df["input_testcases"].iloc[11]

nan

In [52]:
# data_df['input_testcases'] = data_df['input_testcases'].astype(object)
# assert "p02280" in data_df["problem_id"].values
# indices = data_df[data_df['problem_id'] == 'p02280'].index
# print("Indices found:", indices)
# if not indices.empty:
#     # Use the .iat[] or .at[] for a more direct assignment to the first found index
#     data_df.at[indices[0], 'input_testcases'] = tc2input
#     print("Assigned Value:", data_df.at[indices[0], 'input_testcases'])
# data_df.loc[data_df["problem_id"] == "p02280", "input_testcases"] = tc2input
# print(data_df.loc[data_df["problem_id"] == "p02280", "input_testcases"])

Indices found: Index([11], dtype='int64')
Assigned Value: {'0': '8\n0 -1 -1\n1 7 4\n2 -1 -1\n3 -1 -1\n4 0 6\n5 3 -1\n6 -1 -1\n7 5 2', '1': '10\n0 -1 -1\n1 -1 -1\n2 -1 -1\n3 0 9\n4 -1 -1\n5 2 -1\n6 3 5\n7 6 8\n8 1 4\n9 -1 -1', '2': '4\n0 3 2\n1 -1 -1\n2 -1 -1\n3 1 -1', '3': '15\n0 -1 -1\n1 -1 -1\n2 -1 -1\n3 5 8\n4 12 11\n5 10 7\n6 -1 -1\n7 -1 -1\n8 1 2\n9 3 4\n10 -1 -1\n11 6 0\n12 14 13\n13 -1 -1\n14 -1 -1', '4': '10\n0 -1 -1\n1 5 -1\n2 -1 -1\n3 4 1\n4 2 9\n5 -1 -1\n6 3 8\n7 -1 -1\n8 0 7\n9 -1 -1', '5': '18\n0 -1 -1\n1 16 15\n2 12 10\n3 2 1\n4 -1 -1\n5 -1 -1\n6 13 17\n7 -1 -1\n8 3 6\n9 -1 -1\n10 11 -1\n11 -1 -1\n12 0 4\n13 5 7\n14 -1 -1\n15 -1 -1\n16 -1 -1\n17 14 9', '6': '12\n0 -1 -1\n1 -1 -1\n2 9 4\n3 -1 -1\n4 10 8\n5 11 3\n6 2 5\n7 -1 -1\n8 -1 -1\n9 1 0\n10 -1 -1\n11 7 -1', '7': '21\n0 5 14\n1 -1 -1\n2 -1 -1\n3 8 15\n4 6 1\n5 17 13\n6 -1 -1\n7 -1 -1\n8 -1 -1\n9 0 19\n10 -1 -1\n11 -1 -1\n12 10 7\n13 2 20\n14 3 11\n15 -1 -1\n16 -1 -1\n17 18 16\n18 -1 -1\n19 4 12\n20 -1 -1', '8': '15\n0

In [37]:
data_df.loc[data_df["problem_id"] == "p02280"]

,problem_id,description_string,extract_args_fun,tc_gen_fun,input_testcases,output_testcases,num_input_files
11,p02280,### Input Description:\n\nAn integer \( n \) (...,def extract_arguments(fh):\n n = int(fh.rea...,import random\n\ndef generate_binary_tree(n):\...,NaN,NaN,100


In [55]:

# print the three of them, let's make sure they are fixed
data_df = pd.read_json("/home/shypula/program_cal_diverse/data/open_ended_final/dataset_update.jsonl", lines=True, orient="records")
p01508 = data_df[data_df["problem_id"] == "p01508"].iloc[0]
print(p01508["extract_args_fun"])
print('************')
p02051 = data_df[data_df["problem_id"] == "p02051"].iloc[0]
print(p02051["description_string"])
print('************')
p03843 = data_df[data_df["problem_id"] == "p03843"].iloc[0]
print(p03843["extract_args_fun"])
print('************')
p02280 = data_df[data_df["problem_id"] == "p02280"].iloc[0]
print(p02280["input_testcases"]['0'])



def extract_arguments(fh):
    # Read all lines from the file handle
    lines = fh.readlines()
    
    # Initialize an empty list to store all test cases
    test_cases = []
    
    index = 0
    case_number = 1
    
    while index < len(lines):
        # Read the number of vertices
        N = int(lines[index].strip())
        
        if N == 0:
            break
        
        index += 1
        vertices = []
        
        # Read the N vertices
        for _ in range(N):
            x, y = map(int, lines[index].strip().split())
            vertices.append((x, y))
            index += 1
        
        # Append the extracted test case to the list
        test_cases.append((N, vertices))
        case_number += 1
    
    return test_cases


#if __name__ == "__main__":
#    input_path = sys.argv[1]
#    with open(input_path, 'r') as fh:
#        test_cases = extract_arguments(fh)
#    f(test_cases)


#1. **Extracting Arguments**:
#   - `extract_arguments` reads the input file

In [ ]:
# with open("/home/shypula/program_cal_diverse/tc_temp.txt", "w") as f:
#     f.write(p03843["input_testcases"]['0'])
# with open("/home/shypula/program_cal_diverse/tc_temp.txt", "r") as f:
#     print(*extract_arguments(f))

In [14]:
def extract_arguments(fh):
    N = int(fh.readline().strip())
    edges = []
    for _ in range(N - 1):
        a, b = map(int, fh.readline().strip().split())
        edges.append((a, b))
    s = fh.readline().strip()
    return N, edges, s


4 [(1, 2), (2, 3), (1, 4)] 1100


In [21]:
print(data_df[data_df["problem_id"] == "p02280"].iloc[0]["input_testcases"]['50'])

9
9
0 6 7
1 -1 -1
2 -1 -1
3 1 8
4 -1 -1
5 0 3
6 4 2
7 -1 -1
8 -1 -1


In [ ]:



# record["problem_id"] = problem_id
# record["input_testcases"] = tc2input
# record["output_testcases"] = tc2output
# record["num_input_files"] = num_input_files